In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [1]:
import io, sys, math
import numpy as np
from collections import defaultdict
from math import log

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [2]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [3]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [4]:
def softmax(x):
    ### FILL CODE
  max = x.max()
  _exp = np.exp(x - max)
  sum_exp = np.sum(_exp)
  sm = _exp/sum_exp
  return sm

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [5]:
def sgd(w, data, niter):
    nlabels, dim = w.shape
    for iter in range(niter):
      train_err= 0.0

        ### FILL CODE
      for y_i, x_i in data:
        _pred= softmax(np.dot(w, x_i))
        train_err+= math.log(_pred[y_i])
        zero_vec=np.zeros(nlabels)
        zero_vec[y_i]= 1.0
        error= _pred-zero_vec
        gradient= error.reshape(nlabels, 1)*x_i.reshape((1, dim))
        w= w- 0.5*gradient
      print("iter: %02d loss: %.3f" % (iter, train_err/len(data)))
    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [6]:
def predict(w, x):

  ## FILL CODE
  _pred= softmax(np.dot(w, x))
  return np.argmax(_pred)


Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [7]:
def compute_accuracy(w, valid_data):
    ## FILL CODE
    accuracy = 0.0
    for y_i, x_i in valid_data:
      _pred= predict(w, x_i)
      if _pred == y_i:
        accuracy+=1.0
    print(accuracy)
    return (accuracy/float(len(valid_data))) * 100.00

In [8]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("train1.txt")
train_data = load_data("train1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **

iter: 00 loss: -0.491
iter: 01 loss: -0.205
iter: 02 loss: -0.156
iter: 03 loss: -0.131
iter: 04 loss: -0.117

933.0
Validation accuracy: 93.300

